In [ ]:
import numpy as np
from pandas import DataFrame
from pandas import concat
import pandas
import time
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, Dense,SimpleRNN,GRU
from keras import layers
from keras.models import Model

# read and prepare data ################################################################################################################
df = pandas.read_csv('UberDemand.csv')
df1=np.asarray(df[['pickups','spd','vsb','temp','dewp','slp','pcp01','pcp06','pcp24','sd','hday','hr_sin','hr_cos','day_sin','day_cos']])
df2=np.asarray(df[['spd','vsb','temp','dewp','slp','pcp01','pcp06','pcp24','sd','hday','hr_sin','hr_cos','day_sin','day_cos']])

d2=[(df2[4*i,]) for i in range(4343)]
d2=np.reshape(d2,[4343,14])

a=np.zeros((4343, 4))
for i in range (4343):
   a[i,0] =df1[4*i,0]
   a[i,1] =df1[(4*i)+1,0]
   a[i,2] =df1[(4*i)+2,0]
   a[i,3] =df1[(4*i)+3,0]
mydata=np.concatenate((a,d2),1)

# correlogram
import seaborn as sns
sns.pairplot(DataFrame(mydata))
plt.show()
plt.plot(df['hr_sin'],df['hr_cos'])

Encoder

In [ ]:
# Encoder Layer
encoding_dim = 1
x_train=mydata[:,9:12]
x_train = x_train.reshape(433,3)
encoding_dim = 1
input_img = layers.Input(shape=(3,))
encoded = layers.Dense(encoding_dim, activation='relu')(input_img)
decoded = layers.Dense(3, activation='sigmoid')(encoded)
autoencoder_Model = Model(input_img, decoded)
encoder_Model = Model(input_img, encoded)
encoded_input = layers.Input(shape=(encoding_dim,))
decoder_layer = autoencoder_Model.layers[-1]
decoder_Model = Model(encoded_input, decoder_layer(encoded_input))
autoencoder_Model.compile(optimizer='adam', loss='binary_crossentropy')

#  train our autoencoder
n_epochs = 1000
autoencoder_Model.fit(x_train, x_train, epochs=n_epochs, batch_size=256, validation_split=0.2, shuffle=True,)
pcp = encoder_Model.predict(x_train)
pcp /=max(pcp)

mydata=np.delete(mydata,[11,9,10],1)   # dewp removing (dewp and temp have strong correlation)
mydata=np.concatenate((mydata,pcp),1)

In [ ]:
# prepare time and date
time_day=np.zeros((4343, 1))
for i in range (4343):
    time_day[i,0]=i%24
    #time_day[i,0]=(i//24)%7
mydata=np.delete(mydata,[6,8,9,10,12,14,15,16,17],1)  
mydata=np.concatenate((mydata,time_day),1)

# normalize Data
for i in range (mydata.shape[1]): mydata[:,i] /=max(mydata[:,i])

# create train and test data for last 6 hours
mydata = DataFrame(mydata)
data = list()
for i in range(7): data.append(mydata.shift(-6+i))
data = concat(data, axis=1)
data=data.values

X_train = data[:3800, mydata.shape[1]:]
X_train = X_train.reshape((X_train.shape[0], 6, mydata.shape[1]))
Y_train = data[:3800, :4]

X_test = data[3801:4300, mydata.shape[1]:]
X_test = X_test.reshape((X_test.shape[0], 6, mydata.shape[1]))
Y_test = data[3801:4300, :4]

GRU Model

In [ ]:
# network parameter
epochs = 100
batch_size = 50
loss='mae'
optimizer='adam'

# Network
model = Sequential()
model.add(GRU(40, input_shape=(X_train.shape[1], X_train.shape[2]), name="GRU_Layer"))
model.add(Dense(4, name="FC_Layer"))
model.summary()
model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

start = time.clock()
history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=2, shuffle=False)
end = time.clock()

LSTM

In [ ]:
#network parameter
epochs = 100
batch_size = 50
loss='mae'
optimizer='adam'

# Network
model = Sequential()
model.add(LSTM(40, input_shape=(X_train.shape[1], X_train.shape[2]),name="LSTM_Layer"))
model.add(Dense(4, name="FC_Layer"))
model.summary()
model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

start = time.clock()
history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=2, shuffle=False)
end = time.clock()

SimpleRNN

In [ ]:
#network parameter
epochs = 100
batch_size = 50
loss='mae'
optimizer='adam'

# Network
model = Sequential()
model.add(SimpleRNN(40, input_shape=(X_train.shape[1], X_train.shape[2]), name="RNN_Layer"))
model.add(Dense(4, name="FC_Layer"))
model.summary()
model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
 
start = time.clock()
history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=2, shuffle=False)
end = time.clock()

Plotting Results

In [ ]:
print( "In this model " + str(epochs) + " epochs was trained in "  + str((end - start)) + " seconds."
      + " Training loss is " +str(history.history['loss'][-1]) + "and validation loss is " + str(history.history['val_loss'][-1])
      + ". Training accuracy is " +str(history.history['acc'][-1]) + "and validation accuracy is " + str(history.history['val_acc'][-1]))

# plot history
plt.figure()
plt.xlabel('Epochs number')
plt.ylabel('loss') 
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['train loss','validation loss'])
plt.show()

plt.figure()
plt.xlabel('Epochs number')
plt.ylabel('acc') 
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.legend(['train acc','validation acc'])
plt.show()

# Evaluate Model
test_loss, test_acc = model.evaluate(X_test,Y_test)
print( ' Test accuracy is ' + str(test_acc))
print( ' Test loss is ' +  str(test_loss))

training_prediction = model.predict(X_train)
testing_prediction = model.predict(X_test)

# For training data
tittle=["Bronx","Brooklyn","Manhattan","Queens"]
plt.figure(figsize=(10,6))
for i in range (4):
    plt.subplot(2, 2,i+1 )
    plt.tight_layout()
    plt.xlabel('Data nember')
    plt.ylabel('normalize pickups')
    plt.plot(Y_train[:,i],'r',label='train Data')
    plt.plot(training_prediction[:,i], 'b',label='predicted')
    plt.legend()
    plt.title(tittle[i])

plt.figure(figsize=(10,6))
for i in range (4):
    plt.subplot(2, 2,i+1 )
    plt.tight_layout()
    plt.xlabel('Data nember')
    plt.ylabel('normalize pickups')
    plt.plot(Y_test[:,i],'r',label='test Data')
    plt.plot(testing_prediction[:,i], 'b',label='predicted')
    plt.legend()
    plt.title(tittle[i])